# Loading a Trained Neuron Array

In this tutorial we will demonstrate how to load a neuron array directly into the pipeline in order to create pipelines based off of pre-trained models

## Try It Yourself

You can [download the Notebook file here](https://sensiml.com/documentation/_static/file/loading-a-trained-neuron-array.zip) to follow along with this tutorial in your own environment.

In [6]:
import pandas as pd
from sensiml import SensiML

client = SensiML()
client.project ='Activity Case Study'
client.pipeline = 'Activity_Pipeline'

In [7]:
sensor_columns = client.project.columns()
df = pd.read_csv('data/activity_data_medium.csv')
df['Subject'] = df['Subject'].apply(lambda x: int(x[1:]))

client.upload_dataframe('activity_data.csv', df)
df.head()

Uploading file "activity_data.csv" to SensiML Cloud.


,Subject,Activity,AccelerometerX,AccelerometerY,AccelerometerZ
0,1,0,-317,-3000,925
1,1,0,-284,-2968,903
2,1,0,-243,-2987,933
3,1,0,-193,-3051,936
4,1,0,-150,-3059,915


**Neuron Array Format**

The following is the format of a neuron array for a trained model.

In [8]:
neuron_array = [{u'AIF': 1,
              u'Category': 2,
              u'Context': 1,
              u'Identifier': 1,
              u'Vector': [0, 0, 0, 0, 0, 124, 0, 0]},
              {u'AIF': 1,
              u'Category': 2,
              u'Context': 1,
              u'Identifier': 2,
              u'Vector': [166, 31, 172, 138, 93, 31, 254, 35]},
              {u'AIF': 143,
              u'Category': 3,
              u'Context': 1,
              u'Identifier': 3,
              u'Vector': [246, 232, 243, 68, 241, 156, 72, 16]},
              {u'AIF': 1,
              u'Category': 3,
              u'Context': 1,
              u'Identifier': 4,
              u'Vector': [134, 171, 163, 42, 167, 114, 19, 254]},
              {u'AIF': 349,
              u'Category': 4,
              u'Context': 1,
              u'Identifier': 5,
              u'Vector': [118, 165, 147, 184, 156, 191, 2, 0]},
              {u'AIF': 449,
              u'Category': 1,
              u'Context': 1,
              u'Identifier': 6,
              u'Vector': [97, 78, 107, 43, 92, 106, 56, 4]},
              {u'AIF': 354,
              u'Category': 4,
              u'Context': 1,
              u'Identifier': 7,
              u'Vector': [131, 184, 165, 53, 174, 128, 2, 3]}]

**Create a Pipeline**

Create a simple pipeline. 

In [9]:
client.pipeline.reset()
client.pipeline.set_columns(data_columns=['AccelerometerY'], group_columns=['Activity','Subject'], label_column='Activity')

client.pipeline.set_input_data('activity_data.csv', data_columns=['AccelerometerY'])


client.pipeline.add_transform('Windowing')

client.pipeline.add_feature_generator(["Mean", 'Standard Deviation', 'Skewness', 'Kurtosis',
                                    '25th Percentile', '75th Percentile', '100th Percentile',
                                    'Zero Crossing Rate'],
                                    function_defaults = {"columns":[u'AccelerometerY']})

client.pipeline.add_transform('Min Max Scale')

**Load the Neuron Array**

Here we will use the training algorithm "Load Neuron Array" which will load the neurons directly into the hardware simulator and run the cross-fold validation on the model. Note: No training occurs here, we are simply using the provided neurons.

In [10]:
client.pipeline.set_training_algorithm('Load Neuron Array', params = {'neuron_array':neuron_array})

client.pipeline.set_validation_method('Stratified K-Fold Cross-Validation', params={'number_of_folds':5})

client.pipeline.set_classifier('PME', params={"classification_mode":'RBF','distance_mode':'L1'})

client.pipeline.set_tvo()

In [11]:
results, stats = client.pipeline.execute()

Executing Pipeline with Steps:

------------------------------------------------------------------------
 0.     Name: activity_data.csv         		Type: featurefile              
------------------------------------------------------------------------
------------------------------------------------------------------------
 1.     Name: Windowing                 		Type: segmenter                
------------------------------------------------------------------------
------------------------------------------------------------------------
 2.     Name: generator_set             		Type: generatorset             
------------------------------------------------------------------------
------------------------------------------------------------------------
 3.     Name: Min Max Scale             		Type: transform                
------------------------------------------------------------------------
------------------------------------------------------------------------
 4.     Name: t

In [12]:
results.summarize()

TRAINING ALGORITHM: Load Neuron Array
VALIDATION METHOD:  Stratified K-Fold Cross-Validation
CLASSIFIER:         PME

AVERAGE METRICS:
                 F1_SCORE:  13.8  std: 1.60
                PRECISION:  28.5  std: 11.54
              SENSITIVITY:  25.6  std: 0.67

--------------------------------------

STRATIFIED K-FOLD CROSS-VALIDATION MODEL RESULTS : SET VALIDATION

MODEL INDEX: Fold 1
                 F1_SCORE:    train: 13.20  validation: 16.37  
              SENSITIVITY:    train: 25.33  validation: 26.75  

MODEL INDEX: Fold 3
                 F1_SCORE:    train: 13.90  validation: 13.65  
              SENSITIVITY:    train: 25.66  validation: 25.44  

MODEL INDEX: Fold 4
                 F1_SCORE:    train: 13.87  validation: 13.80  
              SENSITIVITY:    train: 25.66  validation: 25.44  

MODEL INDEX: Fold 0
                 F1_SCORE:    train: 13.96  validation: 13.42  
              SENSITIVITY:    train: 25.66  validation: 25.43  

MODEL INDEX: Fold 2
        